In [1]:
import pandas as pd
import numpy as np

#### Parcel File

In [2]:
parcel_data_path = "C:/Users/etheocharides/Box/Modeling and Surveys/Urban Modeling/Bay Area UrbanSim/PBA50/Final Blueprint runs/Final Blueprint (s24)/BAUS v2.25 - FINAL VERSION/"
parcel_data_file = "run182_parcel_data_2035.csv"
parcel_data = pd.read_csv(parcel_data_path+parcel_data_file)

In [3]:
print("number of q1 households is {}".format(parcel_data.hhq1.sum()))
print("number of q2 households is {}".format(parcel_data.hhq2.sum()))

number of q1 households is 973831.0
number of q2 households is 724576.0


In [4]:
num_hh_to_move = 112964

In [5]:
# randomly select a set of parcels
ids_to_move = np.random.choice(parcel_data.loc[parcel_data.hhq1 > 0].parcel_id, num_hh_to_move, replace=False)LOCKED

In [6]:
# make Q1 HHs Q2 HHs 
parcel_data.loc[parcel_data.parcel_id.isin(ids_to_move), "hhq1"] = parcel_data.hhq1 - 1
parcel_data.loc[parcel_data.parcel_id.isin(ids_to_move), "hhq2"] = parcel_data.hhq2.fillna(0) + 1

In [7]:
print("number of q1 households is {}".format(parcel_data.hhq1.sum()))
print("number of q2 households is {}".format(parcel_data.hhq2.sum()))

number of q1 households is 860867.0
number of q2 households is 837540.0


In [8]:
# save a copy of the data in the output folder, manually make it the master file as needed
# don't want to risk overwriting the master file (run182_parcel_data_2035_UBI.csv)
new_parcel_data_file = "run182_parcel_data_2035_UBI_output.csv"
parcel_data.to_csv(parcel_data_path+new_parcel_data_file)

#### TAZ File

In [9]:
taz_data_path = "C:/Users/etheocharides/Box/Modeling and Surveys/Urban Modeling/Bay Area UrbanSim/PBA50/Final Blueprint runs/Final Blueprint (s24)/BAUS v2.25 - FINAL VERSION/"
taz_data_file = "run182_taz_summaries_2035.csv"
taz_data = pd.read_csv(taz_data_path+taz_data_file)

In [10]:
print("number of q1 households is {}".format(taz_data.HHINCQ1.sum()))
print("number of q2 households is {}".format(taz_data.HHINCQ2.sum()))

number of q1 households is 973831.0
number of q2 households is 724576.0


In [11]:
parcel_taz_xwalk_path = "D:/w/bayarea_urbansim/data/2018_10_17_parcel_to_taz1454sub.csv"
parcel_taz_xwalk = pd.read_csv(parcel_taz_xwalk_path)
parcel_taz_xwalk.index = parcel_taz_xwalk.PARCEL_ID.round().astype(int)

In [12]:
# randomly selected set of parcels
ids_to_move_df = pd.DataFrame((ids_to_move), columns=['ids']) 

In [13]:
# add TAZ IDs to the random set of parcels
ids_to_move_df['TAZ'] = 0
ids_to_move_df['TAZ'] = ids_to_move_df.ids.map(parcel_taz_xwalk['ZONE_ID'])

In [14]:
# group the random set of parcels by TAZ
ids_to_move_df['id_count'] = 1
ids_by_taz = ids_to_move_df.groupby(['TAZ']).sum().drop(['ids'], 1)

In [15]:
# make Q1 HHs Q2 HHs in these TAZs
for i in ids_by_taz.index:
    count = ids_by_taz.id_count[i]
    taz_data.loc[taz_data.TAZ == i, 'HHINCQ1'] = taz_data.loc[taz_data.TAZ == i].HHINCQ1 - count
    taz_data.loc[taz_data.TAZ == i, 'HHINCQ2'] = taz_data.loc[taz_data.TAZ == i].HHINCQ2 + count

In [16]:
print("number of q1 households is {}".format(taz_data.HHINCQ1.sum()))
print("number of q2 households is {}".format(taz_data.HHINCQ2.sum()))

number of q1 households is 860867.0
number of q2 households is 837540.0


In [17]:
# save a copy of the data in the output folder, manually make it the master file as needed
# don't want to risk overwriting the master file (run182_taz_summaries_2035_UBI.csv)
new_taz_data_file = "run182_taz_summaries_2035_UBI_output.csv"
taz_data.to_csv(taz_data_path+new_taz_data_file)

#### Superdistrict File

In [18]:
sd_data_path = "C:/Users/etheocharides/Box/Modeling and Surveys/Urban Modeling/Bay Area UrbanSim/PBA50/Final Blueprint runs/Final Blueprint (s24)/BAUS v2.25 - FINAL VERSION/"
sd_data_file = "run182_superdistrict_summaries_2035.csv"
sd_data = pd.read_csv(sd_data_path+sd_data_file)

In [19]:
print("number of q1 households is {}".format(sd_data.hhincq1.sum()))
print("number of q2 households is {}".format(sd_data.hhincq2.sum()))

number of q1 households is 973831
number of q2 households is 724576


In [20]:
taz_sd_xwalk = taz_data[['TAZ', 'SD']].set_index('TAZ')

In [21]:
# add superdistrict IDs to the random set of parcels
ids_to_move_df['SD'] = 0
ids_to_move_df['SD'] = ids_to_move_df.TAZ.map(taz_sd_xwalk['SD'])

In [22]:
# group the random set of parcels by superdistrict
ids_by_sd = ids_to_move_df.groupby(['SD']).sum().drop(['ids'], 1)

In [23]:
# make Q1 HHs Q2 HHs in these superdistricts
for i in ids_by_sd.index:
    count = ids_by_sd.id_count[i]
    sd_data.loc[sd_data.superdistrict == i, 'hhincq1'] = sd_data.loc[sd_data.superdistrict == i].hhincq1 - count
    sd_data.loc[sd_data.superdistrict == i, 'hhincq2'] = sd_data.loc[sd_data.superdistrict == i].hhincq2 + count

In [24]:
print("number of q1 households is {}".format(sd_data.hhincq1.sum()))
print("number of q2 households is {}".format(sd_data.hhincq2.sum()))

number of q1 households is 860867
number of q2 households is 837540


In [25]:
# save a copy of the data in the output folder, manually make it the master file as needed
# don't want to risk overwriting the master file (run182_superdistrict_summaries_2035_UBI.csv)
new_sd_data_file = "run182_superdistrict_summaries_2035_UBI_output.csv"
sd_data.to_csv(sd_data_path+new_sd_data_file)